In [14]:
import ipywidgets as ipw
from IPython.display import display
from threading import Thread
from queue import Queue

# Create queue of when to start/stop recording
messages = Queue()
# Stores audio from microphone
recs = Queue()

record_button = ipw.Button(
    description="record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = ipw.Button(
    description="stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = ipw.Output()

def start_rec(data):
    messages.put(True)

    with output:
        display("OK 3 2 1 let's jam")
        record = Thread(target=rec_mic)
        record.start()

        transcribe = Thread(target=speech_rec, args=(output,))
        transcribe.start()

def stop_rec(data):
     with output:
        messages.get()
        display("tomare")

record_button.on_click(start_rec)
stop_button.on_click(stop_rec)

display(record_button, stop_button, output)

Button(button_style='success', description='record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='stop', icon='stop', style=ButtonStyle())

Output()

    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 946, in run
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/s3/0s1b47rj4nz265rgp1mxz6vc0000gn/T/ipykernel_45413/4208238112.py", line 14, in rec_mic
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyaudio/__init__.py", line 639, in open
    stream = PyAudio.Stream(self, *args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyaudio/__init__.py", line 441, in __init__
    self._stream = pa.open(**arguments)
OSError: [Errno -9998] Invalid number of channels


In [15]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyaudio

p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'ZQE-CAA', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.01, 'defaultLowOutputLatency': 0.0024375, 'defaultHighInputLatency': 0.1, 'defaultHighOutputLatency': 0.011770833333333333, 'defaultSampleRate': 48000.0}
{'index': 1, 'structVersion': 2, 'name': 'MateView GT', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.004583333333333333, 'defaultLowOutputLatency': 0.01, 'defaultHighInputLatency': 0.013916666666666667, 'defaultHighOutputLatency': 0.1, 'defaultSampleRate': 48000.0}
{'index': 2, 'structVersion': 2, 'name': 'MacBook Pro Microphone', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.04505668934240363, 'defaultLowOutputLatency': 0.01, 'defaultHighInputLatency': 0.05521541950113379, 'defaultHighOutputLatency': 0.1, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'MacBook Pro Speakers', 'hostApi': 

In [17]:
CHANNELS = 1
FRAME_RATE = 16000

# Can be adjusted to become fully real-time by importing speech rec modules into this notebook
RECORD_SECONDS = 20

AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2


def rec_mic(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    input=True,
                    rate=FRAME_RATE,
                    input_device_index=0,
                    frames_per_buffer=chunk)

    frames = []

    # Keeps recording as long as there is data on the messages queue
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)

        # Every 20 seconds pass audio data to transcription and restarting recording
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recs.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [18]:
import subprocess
import json
from vosk import Model, KaldiRecognizer
import time 

model = Model(model_name ="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_rec(output):
    while not messages.empty():
        frames = recs.get()
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)['text']

        # cased = subprocess.check_output('python recasepunc/recasepunc.py predit recasepunc/checkpoint', shell=True, text=True, input=text)
        output.append_stdout(text)
        time.sleep(1)
        

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:11:12:13:14:15
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /Users/ryan/.cache/vosk/vosk-model-en-us-0.22/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /Users/ryan/.cache/vosk/vosk-model-en-us-0.22/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:294) Loading words from /Users/ryan/.cache/vosk/vosk-model-en-us-0.22/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo /Users/ryan/.cache/vosk/vosk-model-en-us-

- live translation (t5-base)
- live summarization